In [112]:
import os

import pprint

import pandas as pd
import numpy as np


In [113]:
myVideoPath = '/Users/cudmore/Dropbox/PiE/scrambled'

# make a list of .mp4
myFileList = []
for f in os.listdir(path): 
    # f is file name without path
    if f.startswith(','):
        continue
    if f.endswith('.mp4'):
        baseFilename, fileExtension = os.path.splitext(f)
        eventFile = baseFilename + '.txt'
        eventFile = os.path.join(myVideoPath, eventFile)
        if not os.path.isfile(eventFile):
            eventFile = None
        fileDict = {
            'path': os.path.join(dirName, f),
            'mp4File': os.path.join(myVideoPath, f),
            'eventFile': eventFile
        }
        myFileList.append(fileDict)

for f in myFileList:
    pprint.pprint(f)
    

{'eventFile': '/Users/cudmore/Dropbox/PiE/scrambled/s_1831.txt',
 'mp4File': '/Users/cudmore/Dropbox/PiE/scrambled/s_1831.mp4',
 'path': '/Users/cudmore/Dropbox/PiE/s_1831.mp4'}
{'eventFile': None,
 'mp4File': '/Users/cudmore/Dropbox/PiE/scrambled/s_4022.mp4',
 'path': '/Users/cudmore/Dropbox/PiE/s_4022.mp4'}
{'eventFile': '/Users/cudmore/Dropbox/PiE/scrambled/s_4332.txt',
 'mp4File': '/Users/cudmore/Dropbox/PiE/scrambled/s_4332.mp4',
 'path': '/Users/cudmore/Dropbox/PiE/s_4332.mp4'}
{'eventFile': None,
 'mp4File': '/Users/cudmore/Dropbox/PiE/scrambled/s_5347.mp4',
 'path': '/Users/cudmore/Dropbox/PiE/s_5347.mp4'}
{'eventFile': '/Users/cudmore/Dropbox/PiE/scrambled/s_8729.txt',
 'mp4File': '/Users/cudmore/Dropbox/PiE/scrambled/s_8729.mp4',
 'path': '/Users/cudmore/Dropbox/PiE/s_8729.mp4'}
{'eventFile': '/Users/cudmore/Dropbox/PiE/scrambled/s_9529.txt',
 'mp4File': '/Users/cudmore/Dropbox/PiE/scrambled/s_9529.mp4',
 'path': '/Users/cudmore/Dropbox/PiE/s_9529.mp4'}


In [114]:
# read first line of event file into dict
# example: path=/Users/cudmore/Dropbox/PiE/scrambled/s_1831.mp4,fileName=s_1831.mp4,width=1280.0,height=720.0,aspectRatio=0.56,fps=29.97002997002997,numFrames=53973,numSeconds=1800.9,numEvents=5,videoFileNote=,
def readHeader(eventFile):
    with open(eventFile) as f:
        retDict = {}
        line = f.readline().strip() # .strip() removes '\n'
        parts = line.split(',')
        for part in parts:
            nvArray = part.split('=')
            lhs = nvArray[0]
            if lhs == '':
                continue
            if len(nvArray) > 1:
                rhs = nvArray[1]
            else:
                rhs = None
            retDict[lhs] = rhs
        return retDict
    
# read each event file into:
#    (1) a header dictionary 'header'
#    (2) a pandas dataframe 'df' (sorted by 'frameStart')
for d in myFileList:
    if d['eventFile'] is not None:
        d['header'] = readHeader(d['eventFile'])
        d['df'] = pd.read_csv(d['eventFile'], skiprows=1, header=0)
        d['df'] = d['df'].sort_values('frameStart') # sort rows of df on frameStart
    else:
        d['header'] = None
        d['df'] = None

In [115]:
for f in myFileList:
    pprint.pprint(f, indent=4)


{   'df':                                              index          path    cSeconds  \
5  /Users/cudmore/Dropbox/PiE/scrambled/s_1831.mp4  1.544379e+09  2018-12-09   
0  /Users/cudmore/Dropbox/PiE/scrambled/s_1831.mp4  1.544332e+09  2018-12-09   
2  /Users/cudmore/Dropbox/PiE/scrambled/s_1831.mp4  1.544333e+09  2018-12-09   
1  /Users/cudmore/Dropbox/PiE/scrambled/s_1831.mp4  1.544333e+09  2018-12-09   
3  /Users/cudmore/Dropbox/PiE/scrambled/s_1831.mp4  1.544333e+09  2018-12-09   
4  /Users/cudmore/Dropbox/PiE/scrambled/s_1831.mp4  1.544333e+09  2018-12-09   

      cDate  cTime typeNum  typeStr  frameStart  frameStop  numFrames  sStart  \
5  13:17:22      1       a     3078        3524        445     102.70  117.58   
0  00:14:55      1       a    17056       17078         21     569.10  569.84   
2  00:15:18      2       b    17087       17117         29     570.14  571.14   
1  00:15:01      2       b    17149       17178         28     572.20  573.17   
3  00:15:29      3      

In [116]:
# make a report of duration (sec) of each typeNum (1..9) in each file
eventTypes = np.arange(9) + 1 # [1, 2, 3, ..., 9]
for d in myFileList:
    print(d['path']) # the video file
    if d['df'] is not None:
        print('    ', d['eventFile'])
        print('    eventNum duration(s)')
        df = d['df']
        for eventType in eventTypes:
            mySum = df.loc[df['typeNum'] == eventType, 'numSeconds'].sum()
            print('    ', eventType, mySum)
    else:
          print('    ', 'has no events')

/Users/cudmore/Dropbox/PiE/s_1831.mp4
     /Users/cudmore/Dropbox/PiE/scrambled/s_1831.txt
    eventNum duration(s)
     1 0
     2 0
     3 0
     4 0
     5 0
     6 0
     7 0
     8 0
     9 0
/Users/cudmore/Dropbox/PiE/s_4022.mp4
     has no events
/Users/cudmore/Dropbox/PiE/s_4332.mp4
     /Users/cudmore/Dropbox/PiE/scrambled/s_4332.txt
    eventNum duration(s)
     1 1.27
     2 0.73
     3 0.0
     4 0.37
     5 1.07
     6 0.0
     7 0.0
     8 0.0
     9 0.0
/Users/cudmore/Dropbox/PiE/s_5347.mp4
     has no events
/Users/cudmore/Dropbox/PiE/s_8729.mp4
     /Users/cudmore/Dropbox/PiE/scrambled/s_8729.txt
    eventNum duration(s)
     1 1.31
     2 0.4
     3 0.0
     4 0.0
     5 0.57
     6 0.0
     7 0.0
     8 0.0
     9 1.47
/Users/cudmore/Dropbox/PiE/s_9529.mp4
     /Users/cudmore/Dropbox/PiE/scrambled/s_9529.txt
    eventNum duration(s)
     1 0
     2 0
     3 0
     4 0
     5 0
     6 0
     7 0
     8 0
     9 0


In [117]:
# find events that do not have a chunk
# chunkIndex: absolute chunk index
# rChunkIndex index into random order (From Video App interface)


In [122]:
# i put one in file 1831
for d in myFileList:
    print(d['path']) # the video file
    if d['df'] is not None:
        print('    ', d['eventFile'])
        df = d['df']
        
        # for some reason df['chunkIndex'] is type 'object', convert it
        tmp = pd.to_numeric(df['chunkIndex'], errors='coerce')
        noChunk = tmp.loc[ tmp.isnull() ]
        if noChunk.empty:
            print('    all events have chunks')
        else:
            print('     this event has no Chunk:', noChunk)
        
    else:
          print('    ', 'has no events')

/Users/cudmore/Dropbox/PiE/s_1831.mp4
     /Users/cudmore/Dropbox/PiE/scrambled/s_1831.txt
     this event has no Chunk: 5   NaN
Name: chunkIndex, dtype: float64
/Users/cudmore/Dropbox/PiE/s_4022.mp4
     has no events
/Users/cudmore/Dropbox/PiE/s_4332.mp4
     /Users/cudmore/Dropbox/PiE/scrambled/s_4332.txt
    all events have chunks
/Users/cudmore/Dropbox/PiE/s_5347.mp4
     has no events
/Users/cudmore/Dropbox/PiE/s_8729.mp4
     /Users/cudmore/Dropbox/PiE/scrambled/s_8729.txt
    all events have chunks
/Users/cudmore/Dropbox/PiE/s_9529.mp4
     /Users/cudmore/Dropbox/PiE/scrambled/s_9529.txt
     this event has no Chunk: 8   NaN
Name: chunkIndex, dtype: float64
